#### 转成各种分辨率

In [3]:
size_map = {
    "2160P": "3840x2160",
    "1080P": "1920x1080",
    "720P": "1280x720",
    "540P": "960x540",
    "432P": "768x432",
    "360P": "640x360",
}
classes = ["B", "C", "D", "E", "F"]
sizes = ["360P", "540P", "720P", "1080P"]

In [ ]:
import os
import re
from tqdm import tqdm

orig_root  = "/hdd/VVCSeq"
scale_root = "/hdd/CTC/yuv420p"

for c in classes:
    sub_root  = os.path.join(orig_root, c)
    orig_seqs = list(filter(lambda x: x.endswith(".yuv"), os.listdir(sub_root)))
    
    for orig_seq in tqdm(orig_seqs):
        # BQTerrace_1920x1080_60.yuv, MarketPlace_1920x1080_60fps_10bit_420.yuv
        seq_info = re.search(r"([a-zA-Z]+)_(\d+)x(\d+)_(\d+)", orig_seq)
        seq_name, origW, origH, fps = seq_info.groups()
        
        for size in sizes:
            scale_dir = os.path.join(scale_root, c, size)
            os.makedirs(scale_dir, exist_ok=True)
            
            scaleW, scaleH = size_map[size].split("x")[0], size_map[size].split("x")[1]
            if origW == scaleW and origH == scaleH:
                os.system(f"cp {os.path.join(sub_root, orig_seq)} {os.path.join(scale_dir, orig_seq)}")
            
            scale_seq = orig_seq.replace(origW, scaleW).replace(origH, scaleH)
            
            cmd = (f"ffmpeg -y -s {origW}x{origH} -pix_fmt yuv420p -i {os.path.join(sub_root, orig_seq)} "
                   f"-vf scale={scaleW}x{scaleH} -pix_fmt yuv420p {os.path.join(scale_dir, scale_seq)} &")
            os.system(cmd)